In [4]:
! pip install mlflow boto3 awscli optuna xgboost imbalanced-learn
! pip install optuna

In [5]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/")

In [6]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://reddit-reccomender-bucket/11', creation_time=1763396487435, experiment_id='11', last_update_time=1763396487435, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [7]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [9]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Multimodal naive bayes

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

## Step 2: Train-test split BEFORE sampling

X_train_raw,X_test_raw,y_train,y_test = train_test_split(
        df['clean_comment'],df['category'],
        test_size=0.2,
        random_state=42,
        stratify=df['category']
)

# Step 3: BOW vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000

vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)

X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Step 4: Apply Random Undersampler to train only
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train_vec, y_train)



#Function to log results in MLflow
def log_mlflow(model_name,model,X_train,X_test,y_train,y_test):
    with mlflow.start_run():
        #log the model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_Undersampling_BOW_Trigram")
        mlflow.set_tag("experiment_type", "Algorithm comparision")

        #log algorith name as parameter
        mlflow.log_param("algo_name",model_name)

        #Train model
        model.fit(X_train,y_train)
        y_pred =model.predict(X_test)

        # log accuracy
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric("accuracy",accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators',50,300)
    max_depth = trial.suggest_int('max_depth', 3, 20)  # Maximum depth of the tree
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)  # Minimum samples required to split a node
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)  # Minimum samples required at a leaf node


    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   random_state=42
    )
    
    model.fit(X_train_res, y_train_res)
    preds = model.predict(X_test_vec)

    return accuracy_score(y_test,preds)


# Step 7: Run Optuna for Random Forest, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_rf, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        random_state=42)

    # Log the best model with MLflow, passing the algo_name as "RandomForest"
    log_mlflow("RandomForest", best_model, X_train_res, X_test_vec, y_train_res, y_test)

# Run the experiment for aRndomForest
run_optuna_experiment()




[I 2025-11-18 13:54:24,981] A new study created in memory with name: no-name-8b0653cd-2274-4992-bade-c8388579a0a2
[I 2025-11-18 13:54:26,602] Trial 0 finished with value: 0.6466657575344333 and parameters: {'n_estimators': 207, 'max_depth': 11, 'min_samples_split': 8, 'min_samples_leaf': 15}. Best is trial 0 with value: 0.6466657575344333.
[I 2025-11-18 13:54:27,076] Trial 1 finished with value: 0.606163916541661 and parameters: {'n_estimators': 99, 'max_depth': 5, 'min_samples_split': 16, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.6466657575344333.
[I 2025-11-18 13:54:29,195] Trial 2 finished with value: 0.6429837719896359 and parameters: {'n_estimators': 282, 'max_depth': 9, 'min_samples_split': 9, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.6466657575344333.
[I 2025-11-18 13:54:30,170] Trial 3 finished with value: 0.6245738442656484 and parameters: {'n_estimators': 159, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 13}. Best is trial 0 with value

🏃 View run RandomForest_Undersampling_BOW_Trigram at: http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/#/experiments/11/runs/325d78dcfa9a4e849958d9fd17cea002
🧪 View experiment at: http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/#/experiments/11
